In [28]:
import tkinter as tk
import random
import json
import os

# 游戏设置
WIDTH = 600
HEIGHT = 400
SNAKE_SIZE = 10
SPEED = 100  # 控制游戏速度

# 初始化游戏窗口
root = tk.Tk()
root.title("贪吃蛇")

# 创建一个菜单框架，用于全屏菜单
full_screen_menu = tk.Frame(root, width=WIDTH, height=HEIGHT, bg="black")
full_screen_menu.pack_propagate(False)  # 防止frame的内容自动缩放
full_screen_menu.pack()

# 创建画布和得分标签
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")
score_label = tk.Label(root, text="得分: 0", font=("Arial", 14))

# 游戏设置
snake = [(200, 200), (190, 200), (180, 200)]  # 初始蛇身
snake_direction = "Right"
food = None
score = 0
move_task = None  # 用于存储 move_snake 的定时任务

# 排行榜文件路径
leaderboard_file = "leaderboard.json"
leaderboard = []

# 加载排行榜数据
def load_leaderboard():
    global leaderboard
    if os.path.exists(leaderboard_file):
        with open(leaderboard_file, "r") as file:
            leaderboard = json.load(file)
    else:
        leaderboard = []

# 保存排行榜数据
def save_leaderboard():
    with open(leaderboard_file, "w") as file:
        json.dump(leaderboard, file)

# 更新分数
def update_score():
    score_label.config(text=f"得分: {score}")

# 创建食物
def create_food():
    global food
    food_x = random.randint(0, (WIDTH - SNAKE_SIZE) // SNAKE_SIZE) * SNAKE_SIZE
    food_y = random.randint(0, (HEIGHT - SNAKE_SIZE) // SNAKE_SIZE) * SNAKE_SIZE
    food = (food_x, food_y)
    canvas.create_rectangle(food_x, food_y, food_x + SNAKE_SIZE, food_y + SNAKE_SIZE, fill="red", tags="food")

# 画蛇
def draw_snake():
    canvas.delete("snake")
    for segment in snake:
        x, y = segment
        canvas.create_rectangle(x, y, x + SNAKE_SIZE, y + SNAKE_SIZE, fill="green", tags="snake")

# 蛇移动逻辑
def move_snake():
    global snake, snake_direction, food, score, move_task

    head_x, head_y = snake[0]

    if snake_direction == "Right":
        new_head = (head_x + SNAKE_SIZE, head_y)
    elif snake_direction == "Left":
        new_head = (head_x - SNAKE_SIZE, head_y)
    elif snake_direction == "Up":
        new_head = (head_x, head_y - SNAKE_SIZE)
    elif snake_direction == "Down":
        new_head = (head_x, head_y + SNAKE_SIZE)

    snake = [new_head] + snake[:-1]

    # 判断蛇是否撞到自己或者墙
    if new_head in snake[1:] or not (0 <= new_head[0] < WIDTH and 0 <= new_head[1] < HEIGHT):
        game_over()
        return

    # 判断蛇是否吃到食物
    if new_head == food:
        snake.append(snake[-1])  # 增加蛇的长度
        score += 1
        update_score()
        canvas.delete("food")  # 删除旧的食物
        create_food()  # 创建新的食物

    draw_snake()
    move_task = root.after(SPEED, move_snake)  # 重新设置定时器

# 游戏结束
def game_over():
    global leaderboard

    canvas.create_text(WIDTH / 2, HEIGHT / 2, text="游戏结束", fill="white", font=('Arial', 30), tags="game_over")
    canvas.create_text(WIDTH / 2, HEIGHT / 2 + 40, text=f"最终得分: {score}", fill="white", font=('Arial', 20), tags="game_over")
    
    # 更新排行榜
    leaderboard.append(score)
    leaderboard.sort(reverse=True)  # 降序排列
    
    # 保证排行榜最多有10个得分
    if len(leaderboard) > 10:
        leaderboard = leaderboard[:10]

    # 保存排行榜
    save_leaderboard()

# 控制蛇的方向
def change_direction(event):
    global snake_direction
    if event.keysym == "Left" or event.keysym == "a":
        if snake_direction != "Right":
            snake_direction = "Left"
    elif event.keysym == "Right" or event.keysym == "d":
        if snake_direction != "Left":
            snake_direction = "Right"
    elif event.keysym == "Up" or event.keysym == "w":
        if snake_direction != "Down":
            snake_direction = "Up"
    elif event.keysym == "Down" or event.keysym == "s":
        if snake_direction != "Up":
            snake_direction = "Down"

# 创建重新开始和返回菜单按钮
def show_game_menu():
    # 创建一个菜单
    game_menu = tk.Menu(root)
    root.config(menu=game_menu)

    # 添加菜单项
    game_menu.add_command(label="重新开始", command=restart_game)
    game_menu.add_command(label="返回菜单", command=return_to_menu)

# 返回到全屏菜单
def return_to_menu():
    global move_task, snake, snake_direction, food, score

    # 取消之前的定时器任务
    if move_task:
        root.after_cancel(move_task)

    # 清除画布上的所有元素
    canvas.delete("all")
    # 重置游戏相关变量
    snake = [(200, 200), (190, 200), (180, 200)]  # 重置蛇身
    snake_direction = "Right"
    score = 0
    update_score()
    full_screen_menu.pack()  # 显示全屏菜单
    canvas.pack_forget()  # 隐藏游戏画布
    score_label.pack_forget()  # 隐藏分数标签

# 重新开始游戏
def restart_game():
    global snake, snake_direction, food, score, move_task
    # 删除旧的游戏结束提示文本和食物
    canvas.delete("game_over")
    canvas.delete("food")
    
    # 取消之前的定时器任务
    if move_task:
        root.after_cancel(move_task)
    
    # 重新初始化游戏
    snake = [(200, 200), (190, 200), (180, 200)]
    snake_direction = "Right"
    score = 0
    update_score()
    create_food()
    move_snake()  # 重新开始游戏
    show_game_menu()  # 显示游戏菜单

# 查看排行榜
def show_leaderboard():
    leaderboard_window = tk.Toplevel(root)
    leaderboard_window.title("排行榜")
    leaderboard_window.geometry("300x300")
    
    leaderboard_text = tk.Label(leaderboard_window, text="得分排行榜", font=("Arial", 20))
    leaderboard_text.pack(pady=10)

    # 显示排行榜
    for index, score in enumerate(leaderboard, start=1):
        score_label = tk.Label(leaderboard_window, text=f"{index}. {score}", font=("Arial", 14))
        score_label.pack()

# 显示全屏菜单
def show_full_screen_menu():
    full_screen_menu.pack()
    canvas.pack_forget()
    score_label.pack_forget()

    # 创建开始游戏和退出的按钮
    start_button = tk.Button(full_screen_menu, text="开始游戏", command=start_game, font=("Arial", 20))
    start_button.place(relx=0.5, rely=0.4, anchor="center")
    
    leaderboard_button = tk.Button(full_screen_menu, text="排行榜", command=show_leaderboard, font=("Arial", 20))
    leaderboard_button.place(relx=0.5, rely=0.6, anchor="center")

    quit_button = tk.Button(full_screen_menu, text="退出游戏", command=root.quit, font=("Arial", 20))
    quit_button.place(relx=0.5, rely=0.8, anchor="center")


# 初始化游戏
load_leaderboard()  # 加载排行榜
show_full_screen_menu()

# 绑定键盘事件
root.bind("<Left>", change_direction)
root.bind("<Right>", change_direction)
root.bind("<Up>", change_direction)
root.bind("<Down>", change_direction)
root.bind("a", change_direction)
root.bind("d", change_direction)
root.bind("w", change_direction)
root.bind("s", change_direction)

# 调用 focus_set 来确保键盘事件生效
canvas.focus_set()

# 开始游戏
root.mainloop()


In [30]:
import tkinter as tk
import random
import json
import os

# 游戏设置
WIDTH = 600
HEIGHT = 400
SNAKE_SIZE = 10
SPEED = 100  # 控制游戏速度

# 初始化游戏窗口
root = tk.Tk()
root.title("贪吃蛇")

# 创建一个菜单框架，用于全屏菜单
full_screen_menu = tk.Frame(root, width=WIDTH, height=HEIGHT, bg="black")
full_screen_menu.pack_propagate(False)  # 防止frame的内容自动缩放
full_screen_menu.pack()

# 创建画布和得分标签
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")
score_label = tk.Label(root, text="得分: 0", font=("Arial", 14))

# 游戏设置
snake = [(200, 200), (190, 200), (180, 200)]  # 初始蛇身
snake_direction = "Right"
food = None
score = 0
move_task = None  # 用于存储 move_snake 的定时任务

# 排行榜文件路径
leaderboard_file = "leaderboard.json"
leaderboard = []

# 加载排行榜数据
def load_leaderboard():
    global leaderboard
    if os.path.exists(leaderboard_file):
        with open(leaderboard_file, "r") as file:
            leaderboard = json.load(file)
    else:
        leaderboard = []

# 保存排行榜数据
def save_leaderboard():
    with open(leaderboard_file, "w") as file:
        json.dump(leaderboard, file)

# 更新分数
def update_score():
    score_label.config(text=f"得分: {score}")

# 创建食物
def create_food():
    global food
    food_x = random.randint(0, (WIDTH - SNAKE_SIZE) // SNAKE_SIZE) * SNAKE_SIZE
    food_y = random.randint(0, (HEIGHT - SNAKE_SIZE) // SNAKE_SIZE) * SNAKE_SIZE
    food = (food_x, food_y)
    canvas.create_rectangle(food_x, food_y, food_x + SNAKE_SIZE, food_y + SNAKE_SIZE, fill="red", tags="food")

# 画蛇
def draw_snake():
    canvas.delete("snake")
    for segment in snake:
        x, y = segment
        canvas.create_rectangle(x, y, x + SNAKE_SIZE, y + SNAKE_SIZE, fill="green", tags="snake")

# 蛇移动逻辑
def move_snake():
    global snake, snake_direction, food, score, move_task

    head_x, head_y = snake[0]

    if snake_direction == "Right":
        new_head = (head_x + SNAKE_SIZE, head_y)
    elif snake_direction == "Left":
        new_head = (head_x - SNAKE_SIZE, head_y)
    elif snake_direction == "Up":
        new_head = (head_x, head_y - SNAKE_SIZE)
    elif snake_direction == "Down":
        new_head = (head_x, head_y + SNAKE_SIZE)

    snake = [new_head] + snake[:-1]

    # 判断蛇是否撞到自己或者墙
    if new_head in snake[1:] or not (0 <= new_head[0] < WIDTH and 0 <= new_head[1] < HEIGHT):
        game_over()
        return

    # 判断蛇是否吃到食物
    if new_head == food:
        snake.append(snake[-1])  # 增加蛇的长度
        score += 1
        update_score()
        canvas.delete("food")  # 删除旧的食物
        create_food()  # 创建新的食物

    draw_snake()
    move_task = root.after(SPEED, move_snake)  # 重新设置定时器

# 游戏结束
def game_over():
    global leaderboard

    canvas.create_text(WIDTH / 2, HEIGHT / 2, text="游戏结束", fill="white", font=('Arial', 30), tags="game_over")
    canvas.create_text(WIDTH / 2, HEIGHT / 2 + 40, text=f"最终得分: {score}", fill="white", font=('Arial', 20), tags="game_over")
    
    # 更新排行榜
    leaderboard.append(score)
    leaderboard.sort(reverse=True)  # 降序排列
    
    # 保证排行榜最多有10个得分
    if len(leaderboard) > 10:
        leaderboard = leaderboard[:10]

    # 保存排行榜
    save_leaderboard()

# 控制蛇的方向
def change_direction(event):
    global snake_direction
    if event.keysym == "Left" or event.keysym == "a":
        if snake_direction != "Right":
            snake_direction = "Left"
    elif event.keysym == "Right" or event.keysym == "d":
        if snake_direction != "Left":
            snake_direction = "Right"
    elif event.keysym == "Up" or event.keysym == "w":
        if snake_direction != "Down":
            snake_direction = "Up"
    elif event.keysym == "Down" or event.keysym == "s":
        if snake_direction != "Up":
            snake_direction = "Down"

# 创建重新开始和返回菜单按钮
def show_game_menu():
    # 创建一个菜单
    game_menu = tk.Menu(root)
    root.config(menu=game_menu)

    # 添加菜单项
    game_menu.add_command(label="重新开始", command=restart_game)
    game_menu.add_command(label="返回菜单", command=return_to_menu)

# 返回到全屏菜单
def return_to_menu():
    global move_task, snake, snake_direction, food, score

    # 取消之前的定时器任务
    if move_task:
        root.after_cancel(move_task)

    # 清除画布上的所有元素
    canvas.delete("all")
    # 重置游戏相关变量
    snake = [(200, 200), (190, 200), (180, 200)]  # 重置蛇身
    snake_direction = "Right"
    score = 0
    update_score()
    full_screen_menu.pack()  # 显示全屏菜单
    canvas.pack_forget()  # 隐藏游戏画布
    score_label.pack_forget()  # 隐藏分数标签

# 重新开始游戏
def restart_game():
    global snake, snake_direction, food, score, move_task
    # 删除旧的游戏结束提示文本和食物
    canvas.delete("game_over")
    canvas.delete("food")
    
    # 取消之前的定时器任务
    if move_task:
        root.after_cancel(move_task)
    
    # 重新初始化游戏
    snake = [(200, 200), (190, 200), (180, 200)]
    snake_direction = "Right"
    score = 0
    update_score()
    create_food()
    move_snake()  # 重新开始游戏
    show_game_menu()  # 显示游戏菜单

# 查看排行榜
def show_leaderboard():
    leaderboard_window = tk.Toplevel(root)
    leaderboard_window.title("排行榜")
    leaderboard_window.geometry("300x300")
    
    leaderboard_text = tk.Label(leaderboard_window, text="得分排行榜", font=("Arial", 20))
    leaderboard_text.pack(pady=10)

    # 显示排行榜
    for index, score in enumerate(leaderboard, start=1):
        score_label = tk.Label(leaderboard_window, text=f"{index}. {score}", font=("Arial", 14))
        score_label.pack()

# 显示全屏菜单
def show_full_screen_menu():
    full_screen_menu.pack()
    canvas.pack_forget()
    score_label.pack_forget()

    # 创建开始游戏和退出的按钮
    start_button = tk.Button(full_screen_menu, text="开始游戏", command=start_game, font=("Arial", 20))
    start_button.place(relx=0.5, rely=0.4, anchor="center")
    
    leaderboard_button = tk.Button(full_screen_menu, text="排行榜", command=show_leaderboard, font=("Arial", 20))
    leaderboard_button.place(relx=0.5, rely=0.6, anchor="center")

    quit_button = tk.Button(full_screen_menu, text="退出游戏", command=root.quit, font=("Arial", 20))
    quit_button.place(relx=0.5, rely=0.8, anchor="center")


# 启动游戏
def start_game():
    full_screen_menu.pack_forget()  # 隐藏菜单
    canvas.pack()  # 显示游戏画布
    score_label.pack()  # 显示得分标签
    create_food()  # 创建食物
    move_snake()  # 启动蛇的移动
    show_game_menu()  # 显示游戏菜单（暂停游戏时用）

# 初始化游戏
load_leaderboard()  # 加载排行榜
show_full_screen_menu()

# 绑定键盘事件
root.bind("<Left>", change_direction)
root.bind("<Right>", change_direction)
root.bind("<Up>", change_direction)
root.bind("<Down>", change_direction)
root.bind("a", change_direction)
root.bind("d", change_direction)
root.bind("w", change_direction)
root.bind("s", change_direction)

# 调用 focus_set 来确保键盘事件生效
canvas.focus_set()

# 开始游戏
root.mainloop()
